In [33]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data import get_tokenizer
from tqdm.auto import tqdm

In [34]:
class Corpus(Dataset):
    def __init__(self, data_file_path, window_size=2, tokenizer=None):
        self.window_size = window_size
        self.context_target = []
        with open(data_file_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
        if tokenizer:
            lines = [tokenizer(line.strip()) for line in lines]
        else:
            lines = [line.strip().split(" ") for line in lines]
        self.vocab = build_vocab_from_iterator(lines)
        self.word_to_idx = self.vocab.get_stoi()
        self.idx_to_word = self.vocab.get_itos()
        for line in lines:
            self.context_target.extend(
                [
                    (
                        [line[i - (j + 1)] for j in range(window_size)]
                        + [line[i + (j + 1)] for j in range(window_size)],
                        line[i],
                    )
                    for i in range(window_size, len(line) - window_size)
                ]
            )

    def __getitem__(self, idx):
        context = torch.tensor(self.vocab(self.context_target[idx][0]), dtype=torch.long)
        target = torch.tensor(self.vocab[self.context_target[idx][1]], dtype=torch.long)
        return context, target

    def __len__(self):
        return len(self.context_target)

In [35]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs):
        embeds = torch.mean(self.embedding(inputs), dim=1)
        return self.linear(embeds)

In [36]:
WINDOWS_SIZE = 2
EMBEDDING_DIM = 30
BATCH_SIZE = 512
NUM_EPOCH = 300
NUM_WORKERS = 12
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [37]:
corpus = Corpus('zh.txt', window_size=WINDOWS_SIZE)
# corpus = Corpus('en.txt', window_size=WINDOWS_SIZE, tokenizer=get_tokenizer("basic_english"))
vocab = corpus.vocab

In [38]:
dataloader = DataLoader(corpus, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

In [39]:
model = CBOW(len(corpus.vocab), EMBEDDING_DIM).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [40]:
for epoch in range(NUM_EPOCH):
    total_loss = 0
    correct = 0
    total_num = 0
    for context, target in tqdm(dataloader, leave=False):
        batch_size = context.shape[0]
        total_num += batch_size
        optimizer.zero_grad()
        context, target = context.to(DEVICE), target.to(DEVICE)
        pred = model(context)
        correct += (torch.argmax(pred, dim=1) == target).sum().item()
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch_size
    print(f"Epoch {epoch + 1} Loss: {total_loss / total_num:.4f} Acc: {correct / total_num:.4f}")

Epoch 1 Loss: 7.2787 Acc: 0.0841


Epoch 2 Loss: 6.1037 Acc: 0.1319


Epoch 3 Loss: 5.5928 Acc: 0.1569


Epoch 4 Loss: 5.2003 Acc: 0.1772


Epoch 5 Loss: 4.8770 Acc: 0.1981


...

Epoch 296 Loss: 1.9852 Acc: 0.5818


Epoch 297 Loss: 1.9838 Acc: 0.5829


Epoch 298 Loss: 1.9824 Acc: 0.5828


Epoch 299 Loss: 1.9821 Acc: 0.5821


Epoch 300 Loss: 1.9809 Acc: 0.5829


In [41]:
word_embeddings = {word: model.embedding.weight[idx].tolist() for idx, word in enumerate(corpus.idx_to_word)}

In [42]:
with open('zh_embeddings.txt', 'w') as f:
    for word, embedding in word_embeddings.items():
        f.write(f'{word}: {embedding}\n')

In [43]:
embedding = model.embedding.to('cpu')

In [44]:
vectors = torch.stack([embedding(torch.tensor(vocab[s])) for s in vocab.get_itos()], 0)

In [45]:
def close_words(x, n = 5):
  vec = embedding(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.get_itos()[x] for x in top5 ]

In [48]:
close_words('中国')

['中国', '美国', '日本', '我国', '我们']